# 捨入與截斷誤差 Round-off and Truncation Errors
本章介紹各種數值誤差與他們的意義。數值方法往往必須用有限的位數表示自然界中無限的實數，因此絕對會有誤差。比如說一個64位元的浮點數，最高可以精確到小數點後12位；但是32位元的浮點數就只能精準到小數點後7位。當然之後我們會探討「精準到幾位數」又是如何定義的。
## 誤差 Errors
### Accuracy vs Precision
這兩個字翻成中文都是**精準度**，但是他們在意義上有些許不同。Accuracy 偏向準度，Precision 偏向精度。以打靶為例，當你說一個人打靶的Accuracy很高，表示他可能六發都打在靶上沒有脫靶，但是不見得每發都在靶心附近。如果你說這個人Precision很高，那就是他的彈孔非常集中，但這也不代表這些集中的彈孔會在靶心。以數學來說，high accuracy 暗示平均數接近中心，但標準差不見得小；相對的，high precision表示標準差小，但平均不一定接近中心。
![accuracy_vs_precision](http://cdn.antarcticglaciers.org/wp-content/uploads/2013/11/precision_accuracy.png)

### 絕對誤差 Absolute Error
絕對誤差描述的是量測跟真實數值的差異。還記得高中物理做量測的時候嗎。通常量測的數據會有準確項跟估測項。跟那時候一樣，真正的數值就是你能量測到的準確項再加一個誤差
$$
true\ value = measured\ value + error
$$
經過移項就能得到絕對誤差的公式
$$
absolute\ error = {true\ value - measured\ value}
$$

### 相對誤差 Relative Error
當你的真實數值跟你的量測數值都很大的時候，些微的偏移會有看似很大的誤差，但是可能在實際情況下並不是這一回事。比如說莫耳數，$6.02\times10^{23}$，每次加一兩滴液體到容器內就是增加十的30幾次方，絕對誤差看起來就高到爆，但是藥劑實際上偏差並不大。這就是用相對誤差的時機。簡單來說就是絕對誤差除上真實數值已達到歸一化(normalize)的作用。常見的相對誤差通常會乘上100%讓他變成 percent relative error
$$
percent\ relative\ error\ (\epsilon_t) = {{{true\ value - measured\ value} \over {true\ value}} \times 100\%}
$$

當然世間萬物並非完美。有時候你稱為true value的數值根本不能算是真的數值（例如：$\pi$或是$e$這種無限非循環實數）那我們就只能盡可能的用近似值去推估
$$
percent\ relative\ error\ (\epsilon_a) = {{{true\ value - measured\ value} \over {approximation}} \times 100\%}
$$

有些更糟的情況下，有些演算法只能經過迭代(iteration)得到近似值。這種情況下就只能根據現在的數值去推測誤差
$$
percent\ relative\ error\ (\epsilon_a) = {{{current\ approximation - previous\ approximation} \over {current\ approximation}} \times 100\%}
$$

這些迭代式估測當然不能一直迭代下去，通常我們會設定一個終止條件(stopping criteria)讓他在我們認為容許的誤差範圍內就停止。

> Example 4.1 Error Estimates for Iterative Methods  
exponential 常數可以用下列的無窮迴圈表示
$$
e^x = 1 + x + {{x^2}\over 2} + {{x^3}\over 3!} + \dotsb + {{x^n}\over n!}
$$
因為 `numpy.exp` 也只是一個近似，所以我們可以試著用迭代想辦法近似到 `numpy.exp` 的數值

In [1]:
import numpy as np

x = 1
ref_value = np.exp(x)
approx_exp = 1

n = 1
while not np.allclose(approx_exp, ref_value, rtol=1e-5):
    # approx_exp += x**n/np.prod(np.arange(1, n+1))
    approx_exp += x**n/np.math.factorial(n)
    print(n, approx_exp, ref_value)
    n += 1
    
percent_rel_err = (ref_value - approx_exp) / (ref_value) * 100
print(f'percent relative error of estimate: {percent_rel_err:.5f}%')

1 2.0 2.718281828459045
2 2.5 2.718281828459045
3 2.6666666666666665 2.718281828459045
4 2.708333333333333 2.718281828459045
5 2.7166666666666663 2.718281828459045
6 2.7180555555555554 2.718281828459045
7 2.7182539682539684 2.718281828459045
8 2.71827876984127 2.718281828459045
percent relative error of estimate: 0.00011%


> np.allclose 的close指的是接近。我們利用 while 迴圈檢查第幾項之後才達到我們設定的近似值。allclose 的 `rtol` 參數就是在設定我們的終止條件。你可以試著調整 `rtol` 觀察我們必須花多少項(n)才能達到我們設定的精度

## 捨入誤差 Round-off Errors
前面有提到，當我們把現實世界數位化之後，我們能表示的數字就被電腦所能表達的數字長度所限制。  
以尺為例，如果你的尺最小刻度是1mm=0.001m，那你能量到的最小值就是0.001。這支尺沒有辦法表示0.0015，只能將這個數值表示成0.001或是0.002。這個情況就叫做*underflow*也就是我們的精度已經超出我能表示的範圍了。  
另一個情況是，如果你的尺只有30cm，你永遠無法量超過30公分的蔬菜水果。這種情況就被稱為*overflow*。

### 機器誤差 Machine Epsilon
除了上述兩種情況之外，還有另一個比較特殊的情況。為了讓我們每個數字都能用相同的位數表示，所以會用科學符號來代表。比如說在 IEEE 754 的規範中，就規定雙精度浮點數(double precision float)在記憶體中的排列必須是: `1位元 正負號(sign) + 11位元 次方數(exponent) + 52位元 尾數(mantissa)`。為了讓大家了解，我們來假設一個基本的10進位系統
$$
s_1 d_1.d_2 \times 10^{s_0 d_0}
$$
$s_1$ 表示尾數的正負號，$d_1$ 表示整數部份，$d_2$ 表示小數部份；$s_0$ 表示次方的正負號，$d_0$ 表示次方的值。用最基本的尾數兩位、次方介於-2到+2之間來描述的話，我們能量測到的極值為 $\pm9.99\times 10^{+2}$ 也就是 -999 跟 999，超出這個範圍的就是overflow。能量到最趨近於0的數值為 $\pm0.01 \times 10^{-2}$ 也就是 -0.0001 跟 0.0001，在這個數值中間的都會underflow - 無法表示。接下來我們來看正的數列。在 $0.01 \times 10^{-2}$ 到 $1.00 \times 10^{-2}$ 之間我們的每次步進(step)都是0.0001, 但是如果你一到 $0.01 \times 10^{-1}$ 跟 $1.00 \times 10^{-1}$ 的區間你每個間隔都變成0.01，這就表示你的次方離最小值越來越遠就會有更多無法表示得空洞(hole)。而且在$\lt1.00 \times 10^{-2}$的區間內我們能遇到的最大相對誤差會落在step大小一半的位置
$$
\frac{1.00 - 0.99}{2} \times 10^{-2} = 0.00005
$$

0.00005 就是我們的機器誤差($\epsilon$)。
在 numpy 你可以用finfo (讀作f-info) 還獲得當下型別的機器誤差。

In [6]:
print(f'machine epsilon for a {np.finfo(np.float64).bits}-bit float (compliant to IEEE 754) {np.finfo(np.float64).eps}')
print(f'{np.finfo(np.float64).maxexp}')

machine epsilon for a 64-bit float (compliant to IEEE 754) 2.220446049250313e-16
1024


大多數的語言對機器誤差的定義跟要求都是相同的，也就是四則運算後的結果通常是 $0.5\epsilon$。但是在比較舊的函式庫 (i.e. BLAS) 經過四則運算後只能保證 $1\epsilon$ 所以使用上要特別注意。

你可能會問，不會阿我永遠不會用到 BLAS (Basic Linear Algebra Subroutines) 不知道沒關係。但是實際上在使用C/C++做複雜科學運算的時候，有時候會用BLAS做結合。因為 BLAS 是經過數十年最佳化的矩陣運算函式，所以有時候非用不可。

## 截斷誤差 Trucation Error
截斷誤差就是數學運算過程中經過近似所產生的誤差。最明顯的例子就是微分
$$
\frac{dv}{dt} \approx \frac{\Delta v}{\Delta t}
$$
當你的$\Delta t$不夠小的時候，割線跟切線會有一定的誤差。另一個例子是泰勒序列 (Taylor Series)。雖然他能近似任何連續函數，但是必須要有無限項，因此必定會有截斷誤差。

## 總數值誤差 Total Numerical Error
一樣用微分為例。上面提到$\Delta t$必須趨近於零才能讓割線等於切線。但是太小又會讓$\Delta t$本身underflow或是運算數值小於機器誤差。此外還有電腦資源的限制，因此這一切都是工程師需要考慮的trade-off。